<a href="https://colab.research.google.com/github/wshuyi/demo-tabular-customer-churn-tensorflow2-with-normalization/blob/master/demo_tabular_customer_churn_tensorflow2_with_normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://raw.githubusercontent.com/wshuyi/demo-customer-churn-ann/master/customer_churn.csv

In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('customer_churn.csv')

In [0]:
df.head()

In [0]:
df.columns

In [0]:
!pip install -q tensorflow==2.0.0-alpha0

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow import feature_column

In [0]:
tf.random.set_seed(1)

In [0]:
train, test = train_test_split(df, test_size=0.2, random_state=1)

In [0]:
train, valid = train_test_split(train, test_size=0.2, random_state=1)

In [0]:
print(len(train))
print(len(valid))
print(len(test))

In [0]:
feature_columns = []

In [0]:
numeric_columns = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [0]:
train.dtypes

In [0]:
for header in numeric_columns:
  feature_columns.append(
      feature_column.numeric_column(
          header, 
          normalizer_fn=lambda x: (tf.cast(x, dtype=float)-train[header].mean())/train[header].std()))

In [0]:
categorical_columns = ['Geography', 'Gender', 'HasCrCard', 'IsActiveMember']

In [0]:
def get_one_hot_from_categorical(colname):
  categorical = feature_column.categorical_column_with_vocabulary_list(colname, train[colname].unique().tolist())
  return feature_column.indicator_column(categorical)

In [0]:
geography = get_one_hot_from_categorical('Geography'); geography

In [0]:
for col in categorical_columns:
  feature_columns.append(get_one_hot_from_categorical(col))

In [0]:
feature_columns

In [0]:
from tensorflow.keras import layers

In [0]:
feature_layer = layers.DenseFeatures(feature_columns); feature_layer

In [0]:
def df_to_tfdata(df, shuffle=True, bs=32):
  df = df.copy()
  labels = df.pop('Exited')
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(df), seed=1)
  ds = ds.batch(bs)
  return ds

In [0]:
train_ds = df_to_tfdata(train)
valid_ds = df_to_tfdata(valid, shuffle=False)
test_ds = df_to_tfdata(test, shuffle=False)

In [0]:
model = keras.Sequential([
  feature_layer,
  layers.Dense(200, activation='relu'),
  layers.Dense(100, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
model.fit(train_ds,
          validation_data=valid_ds,
          epochs=5)

In [0]:
model.summary()

In [0]:
model.evaluate(test_ds)

In [0]:
pred = model.predict(test_ds)

In [0]:
pred = np.rint(pred)

In [0]:
pred

In [0]:
from sklearn.metrics import classification_report, confusion_matrix

In [0]:
print(classification_report(test['Exited'], pred))

In [0]:
print(confusion_matrix(test['Exited'], pred))

In [0]:
len(test[test['Exited'] == 0])/len(test)

In [0]:
np.unique(pred)